# Deep neural networks for segmentation of cardiac MR images
<span style="font-size:9pt;">
author: MWLafarge (m.w.lafarge@tue.nl); affiliation: Eindhoven University of Technology; created: Feb 2020
</span>

## Preliminaries
In the previous exercises, we saw how to tackle a classification task, by modeling the target likelihood function by a __deep convolutional neural network__.
In this exercise we will address a segmentation problem for which we want to train a deep learning model to produce a binary mask that isolate a specific structure of interest (in this case, LV
endocardial contour). 

### Part 1
The first part of the exercise consists in implementing a baseline model to solve the task at hand.
You will be shown how to implement a popular convoluational network architecture called ["U-net" (Ronneberger et al.)](https://arxiv.org/abs/1505.04597) with __tf.keras__.
This architecture enables to produce dense predictions and is thus very well suited to solve segmentation tasks.

The implementation of this model will introduce the use of __up-sampling layers__, __concatenation layers__ and how to customize a loss function within the __tf.keras__ framework. You will learn how to evaluate the predicted segmentation masks on a hold-out validation set.


### Part 2
The second part of the exercise leaves you free to expend this baseline model in order to improve its performances. By the end of the session, you are expected to process a hold-out test set of the dataset for which the ground-truth masks will be kept secret.

The organizers will evaluate the participant predictions at the end of the session, this will give you a glance of a real-world situation in which your trained model is applied on new unseen data.

## Import the required libraries

In [ ]:
# environement setup
%load_ext autoreload
%autoreload 2
%matplotlib widget
#%matplotlib inline

# system libraries
import sys
import os

# computational libraries
import numpy as np
import tensorflow as tf

# utility functions for this exercise
from utils_ex4 import plot_image_list, plot_image_batch, plot_image_sequence, plot_featureMaps, Monitoring, submit_results

## Dataset description

The dataset of this exercise consist of functional cardiac MR images.
The images were collected from a cohort of 50 patients, and were acquired in different imaging centers (involving different MRI scanners).
These functional images are __short time sequences__ which show the heart motion and its different phases during the cardiac cycle.

The images were registered based on their DICOM format of origin.
The dataset is restricted to the __long axis__ view and includes __2-Chamber, 3-Chamber and 4-Chamber__ views.
The LV endocardial contour was manually annotated such that all images have a binary mask are associated with them (contour:1, otherwise:0).


For the purpose of this exercise, the dataset was split in 3 sets:
- A __training set__ that consists of 28 patients with a total of 2776 frames.
- A __validation set__ that consists of 10 patients with a total of 1330 frames.
- A __test set__ that consists of 12 patients with a total of 1193 frames.

All images are gray-scale in an 8-bit format, with their intensity rescaled to the range \[0, 255\]. Images were resampled to the shape of \[352px, 352px\]. All images of the training set and validation set where perturbed with random spatial transformations.

Each image/mask pair has a unique __id__, that is available in a metadata list given for each dataset split. Each __id__ can be used to fetch the images/mask from their paths, and to access information such as the corresponding __patient id__, __chamber view__ or __index within the functional sequence__.

In [ ]:
PATH_BASE_DIR = os.getcwd() # current working directory

# source paths
PATH_BASE_DIR = "/data/industry_course/CMR"

DATA_PATH_META_TRAIN   = PATH_BASE_DIR + os.sep + "training/metadata.npz"
DATA_PATH_IMAGES_TRAIN = PATH_BASE_DIR + os.sep + "training/images/{id}.npy"
DATA_PATH_MASKS_TRAIN  = PATH_BASE_DIR + os.sep + "training/contourMasks/{id}.npy"

DATA_PATH_META_VALID   = PATH_BASE_DIR + os.sep + "validation/metadata.npz"
DATA_PATH_IMAGES_VALID = PATH_BASE_DIR + os.sep + "validation/images/{id}.npy"
DATA_PATH_MASKS_VALID  = PATH_BASE_DIR + os.sep + "validation/contourMasks/{id}.npy"

DATA_PATH_META_TEST   = PATH_BASE_DIR + os.sep + "test/metadata.npz"
DATA_PATH_IMAGES_TEST = PATH_BASE_DIR + os.sep + "test/images/{id}.npy"
DATA_PATH_MASKS_TEST  = PATH_BASE_DIR + os.sep + "test/contourMasks/{id}.npy"


# import metadata
train_metadata = np.load(DATA_PATH_META_TRAIN)
valid_metadata = np.load(DATA_PATH_META_VALID)
test_metadata  = np.load(DATA_PATH_META_TEST)


# split metadata
train_imageIds_list     = train_metadata["global_id"]
train_patientIds_list   = train_metadata["patient_id"]
train_chamberNames_list = train_metadata["chamber_name"]
train_sequenceIds_list  = train_metadata["sequence_idx"]

valid_imageIds_list     = valid_metadata["global_id"]
valid_patientIds_list   = valid_metadata["patient_id"]
valid_chamberNames_list = valid_metadata["chamber_name"]
valid_sequenceIds_list  = valid_metadata["sequence_idx"]

test_imageIds_list     = test_metadata["global_id"]
test_patientIds_list   = test_metadata["patient_id"]
test_sequenceIds_list  = test_metadata["sequence_idx"]


# check
print("Training set: #patients = {}; #images = {}".format(np.unique(train_patientIds_list).shape[0], train_imageIds_list.shape[0]))
print("Validation set: #patients = {}; #images = {}".format(np.unique(valid_patientIds_list).shape[0], valid_imageIds_list.shape[0]))
print("Test set: #patients = {}; #images = {}".format(np.unique(test_patientIds_list).shape[0], test_imageIds_list.shape[0]))

First, let's visualize a few examples.  As it can be seen, a binary mask is associated with each training and validation image.

In [ ]:
# generate a random list of image indices to fetch
rdm_indices = [train_imageIds_list[np.random.randint(train_imageIds_list.shape[0])] for _i in range(8)]

# scan the list of indices and import image-mask pairs
batch_images = []
batch_masks  = []
for idx_c in rdm_indices:
    img_path  = DATA_PATH_IMAGES_TRAIN.format(id=idx_c)
    mask_path = DATA_PATH_MASKS_TRAIN.format(id=idx_c)
    batch_images.append(np.load(img_path))
    batch_masks.append(np.load(mask_path))

# finally we visualize the imported images
plot_image_batch(np.stack(batch_images), np.stack(batch_masks))

# PART 1: TRAINING A BASELINE U-NET MODEL

Let's recap the task at hand: we want to create a model that takes actual tensor images of shape \[352,352,1\] (images with a single gray-level channel) as input and outputs the pixel-wise likelihood of the presence of the *LV
endocardial contour* (i.e. a probability map).

### Defining the graph operations
For this problem we want to make a pixel-to-pixel prediction. Having a patch-based approach would be one solution (input: image patch, output: prediction of its center pixel), but it is not efficient as it will require to exhaustively scan all possible patches to make a dense prediction (a predicted mask with a pixel-to-pixel mapping).

[Ronneberger et al.](https://arxiv.org/abs/1505.04597) proposed a solution to this problem by the means of a specific network architecture: using skip-connections and up-sampling layers enables to output prediction with a pixel-to-pixel mapping for a large region of the input image. For more details, [see original paper](https://arxiv.org/abs/1505.04597).

Implementing the architecture of a U-net can be done in 3 steps:
- 1) constructing a __down-sampling branch__ of convolutional layers and max-pooling layers
- 2) constructing an __up-sampling branch__ of convolutional layers and un-pooling layers towards a sigmoid-activated output
- 3) concatenating the intermediate features maps of the __down-sampling branch__ to the inputs of the __up-sampling branch__.

The required operations are:
- an __tf.keras.Input()__ placeholder that takes image tensors of size \[352, 352, 1\] as input.
- several [__tf.keras.layers.Conv2D()__](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D) layers to filter intermediate feature maps (convolution operation + non-linearity).
- several [__tf.keras.layers.MaxPool2D()__](https://www.tensorflow.org/api_docs/python/tf/keras/layers/MaxPool2D) layers to down-sample the intermediate feature maps in the __down-sampling branch__.
- several [__tf.keras.layers.UpSampling2D()__](https://www.tensorflow.org/api_docs/python/tf/keras/layers/UpSampling2D) layers to up-sample the intermediate feature maps in the __up-sampling branch__.
- [__tf.keras.layers.Cropping2D()__](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Cropping2D) layers so that the shape of the concatenated intermediate feature maps match the shape of the inputs in the __up-sampling branch__.
- [__tf.keras.layers.Concatenate()__](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Concatenate) layers to connect the __down-sampling branch__ to the __up-sampling branch__.

As for the previous exercise, we will use a regularizer to prevent overfitting.
It is *very* important to keep track of the change of shape of the features maps along the sequence of layers. This way, we make sure we know how to crop and concatenate the feature maps, and we make sure we know the expected shape of the output.


In [ ]:
# placeholder for the image data
image_size = [352, 352, 1]
regularizer = None #tf.keras.regularizers.l2(l=0.0005)

# model hyper-parameters
N = 8 # number of feature maps in each convolutional layer
M = 16 # number of feature maps in the last layer (prior to the output)

# utilitary layers
activation  = "relu"

# input placeholder
inputs = tf.keras.Input(shape=image_size)  #-- Placeholder for gray-scale input images


# down-sampling paths
conv_down1 = tf.keras.layers.Conv2D(
    filters     = N,
    kernel_size = 5,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)
maxPoolLayer1 = tf.keras.layers.MaxPool2D(
    pool_size = (2, 2),
    strides   = None,
    padding   = "valid")

conv_down2 = tf.keras.layers.Conv2D(
    filters     = N,
    kernel_size = 3,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)
maxPoolLayer2 = tf.keras.layers.MaxPool2D(
    pool_size = (2, 2),
    strides   = None,
    padding   = "valid")

conv_down3 = tf.keras.layers.Conv2D(
    filters     = N,
    kernel_size = 3,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)
maxPoolLayer3 = tf.keras.layers.MaxPool2D(
    pool_size = (2, 2),
    strides   = None,
    padding   = "valid")

conv_down4 = tf.keras.layers.Conv2D(
    filters     = N,
    kernel_size = 3,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)
maxPoolLayer4 = tf.keras.layers.MaxPool2D(
    pool_size = (2, 2),
    strides   = None,
    padding   = "valid")

conv_down5 = tf.keras.layers.Conv2D(
    filters     = N,
    kernel_size = 3,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)
upPoolLayer5  = tf.keras.layers.UpSampling2D(
    size          = (2, 2),
    interpolation = "nearest")

# up-sampling path
conv_up4 = tf.keras.layers.Conv2D(
    filters     = N,
    kernel_size = 3,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)
upPoolLayer4  = tf.keras.layers.UpSampling2D(
    size          = (2, 2),
    interpolation = "nearest")

conv_up3 = tf.keras.layers.Conv2D(
    filters     = N,
    kernel_size = 3,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)
upPoolLayer3  = tf.keras.layers.UpSampling2D(
    size          = (2, 2),
    interpolation = "nearest")

conv_up2 = tf.keras.layers.Conv2D(
    filters     = N,
    kernel_size = 3,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)
upPoolLayer2  = tf.keras.layers.UpSampling2D(
    size          = (2, 2),
    interpolation = "nearest")

conv_up1 = tf.keras.layers.Conv2D(
    filters     = M,
    kernel_size = 3,
    strides     = (1, 1),
    padding     = "valid",
    activation  = activation,
    kernel_regularizer = regularizer)

# output layer
layerOut = tf.keras.layers.Conv2D(
    filters     = 1,
    kernel_size = 1,
    strides     = (1, 1),
    padding     = "valid",
    activation  = "sigmoid",
    kernel_regularizer = regularizer)


# utilitary layers
cropLayer1 = tf.keras.layers.Cropping2D(
    cropping    = (44, 44),
    input_shape = (348, 348, N))
concatLayer1  = tf.keras.layers.Concatenate()

cropLayer2 = tf.keras.layers.Cropping2D(
    cropping    = (20, 20),
    input_shape = (172, 172, N))
concatLayer2  = tf.keras.layers.Concatenate()

cropLayer3 = tf.keras.layers.Cropping2D(
    cropping    = (8, 8),
    input_shape = (84, 84, N))
concatLayer3  = tf.keras.layers.Concatenate()

cropLayer4 = tf.keras.layers.Cropping2D(
    cropping    = (2, 2),
    input_shape = (40, 40, N))
concatLayer4  = tf.keras.layers.Concatenate()

### Connecting the graph and instantiating the model
We can now join the graph components to define the model __output__ as a function of the __input placeholder__.

In [ ]:
# graph construction
# down path

x = conv_down1(inputs)
x1_skip = cropLayer1(x)
x = maxPoolLayer1(x)

x = conv_down2(x)
x2_skip = cropLayer2(x)
x = maxPoolLayer2(x)

x = conv_down3(x)
x3_skip = cropLayer3(x)
x = maxPoolLayer3(x)

x = conv_down4(x)
x4_skip = cropLayer4(x)
x = maxPoolLayer4(x)

# bottleneck
x = conv_down5(x)
x = upPoolLayer5(x)

# up path
x = concatLayer4([x4_skip, x])
x = conv_up4(x)
x = upPoolLayer4(x)

x = concatLayer3([x3_skip, x])
x = conv_up3(x)
x = upPoolLayer3(x)

x = concatLayer2([x2_skip, x])
x = conv_up2(x)
x = upPoolLayer2(x)

x = concatLayer1([x1_skip, x])
x = conv_up1(x)

# output
outputs = layerOut(x)

# model definition
model = tf.keras.Model(inputs, outputs)

## Custom weighted cross-entropy

In this dataset, the pixel-wise instances are imbalanced (we have far less foreground pixels than background pixels). If we use a typical cross-entropy loss, all pixel instances will have the same weight, giving more importance to the classification of the background pixel than for the foreground pixels.
A solution consists in __customizing__ the cross-entropy loss (to make a weighted binary cross-entropy loss) such that we calculate and apply a coefficient to artificially re-balance the classes.

In [ ]:
# Home-made loss
def wBCE(y_true, y_pred):
    y_true = tf.keras.backend.cast(y_true, y_pred.dtype)
    
    nb_positive = tf.keras.backend.sum(y_true)
    nb_negative = tf.keras.backend.sum(1.0 - y_true)
    
    freq_positive = (nb_positive + nb_negative) / nb_positive
    freq_negative = (nb_positive + nb_negative) / nb_negative
    
    epsilon = 1e-6
    cross_entropies_positive = -1.0 * y_true * tf.keras.backend.log(y_pred + epsilon)
    cross_entropies_negative = -1.0 * (1.0 - y_true) * tf.keras.backend.log(1.0 - y_pred + epsilon)
    
    cross_entropies_w = freq_positive * cross_entropies_positive + freq_negative * cross_entropies_negative
    cross_entropies_w = 0.5 * cross_entropies_w
    
    return tf.keras.backend.mean(cross_entropies_w)

### Preparing to train the model

Let's define an optimizer, here [Adam with default parameters works fine (see documentation)](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam)).

In [ ]:
# stochastic gradient descent with momentum
# optimizer = tf.keras.optimizers.SGD(
#     learning_rate = 0.01,
#     momentum      = 0.9)

# Adam optimizer
optimizer = tf.keras.optimizers.Adam(
    learning_rate = 0.001,
    beta_1        = 0.9,
    beta_2        = 0.999,
    epsilon       = 1e-07)

### Compiling the model
We finally configure the model for training by indicating the loss, the optimizer, and check the model architecture we implemented.

In [ ]:
model.compile(
    optimizer = optimizer,
    loss      = wBCE,
    metrics   = None)

model.summary()

### Creating a data generator

Again, for memory-wise and training efficiency reasons, we will define generators that can be called to produce __mini-batches__ of samples when needed.

Images are sampled randomly withing their split, they keep their tensor shape, and we will rescale their intensity to be in \[0,1\].   
Note that the masks need to be cropped in order to match the shape of the network output.

In [ ]:
class dataGenerator(tf.keras.utils.Sequence):

    def __init__(self,
                 data_images_ids,
                 batchSize=1,
                 path_images_formated = DATA_PATH_IMAGES_TRAIN,
                 path_masks_formated  = DATA_PATH_MASKS_TRAIN):
        
        self.batch_size = batchSize
        self._path_images_formated = path_images_formated
        self._path_masks_formated  = path_masks_formated
        
        self.data_images_ids = data_images_ids
        self.data_size       = data_images_ids.shape[0]
        self.data_indices_shuffle = np.arange(self.data_size)
        np.random.shuffle(self.data_indices_shuffle)
        
        self.scan_idx = 0

    def __len__(self):
        return int(self.data_size / self.batch_size)

    def __getitem__(self, idx):
        batch_images  = []
        batch_masks   = []
        
        for _i in range(self.batch_size):
            idx_local = self.data_indices_shuffle[self.scan_idx]
            img_id_c = self.data_images_ids[idx_local]
            
            img_path  = self._path_images_formated.format(id=img_id_c)
            mask_path = self._path_masks_formated.format(id=img_id_c)
            
            img_c  = np.load(img_path)
            mask_c = np.load(mask_path)

            batch_images.append(img_c)
            batch_masks.append(mask_c)
        
            self.scan_idx += 1
            self.scan_idx %= self.data_size
            
        batch_images = np.stack(batch_images, axis=0)
        batch_images = np.expand_dims(batch_images, axis=3)
        
        slice_crop  = slice(47, -47, 1) #! The target mask need to be cropped to match the shape of the output of the network
        batch_masks = np.stack(batch_masks, axis=0)
        batch_masks = batch_masks[:, slice_crop, slice_crop]
        batch_masks = np.expand_dims(batch_masks, axis=3)
        
        batch_weights = batch_masks
        
        batch_images = batch_images / 255.0 #-- [0,1] scaling
        return batch_images, batch_masks

Then, we can instantiate 2 generators: one to generate mini-batches of the training data, one to generate mini-batches of the validation data.

In [ ]:
dataGenerator_train = dataGenerator(train_imageIds_list, batchSize=2)
dataGenerator_valid = dataGenerator(valid_imageIds_list[:10], batchSize=10,
        path_images_formated = DATA_PATH_IMAGES_VALID,
        path_masks_formated  = DATA_PATH_MASKS_VALID)

Finally lets check what keras receives when the generator are called:

In [ ]:
arbitrary_iteration_idx = 42
train_images_batch, train_masks_batch = dataGenerator_train[arbitrary_iteration_idx]

print("Mini-batch of images has shape: ", train_images_batch.shape)

## Training the model and monitoring the training process

Now that our generators are instantiated, we can train our model. We will use __tf.keras.Model.fit\_generator__ function to start the training procedure by feeding the data generators.   
To get a better insight and to keep track of the status of the learning process, the Monitoring callback for this exercise will show the predicted mask of a pre-selected validation image.

Let's import a random validation image and instanciate the monitoring callback.

In [ ]:
# select a validation image/mask
img_id_c  = valid_imageIds_list[0]
img_path  = DATA_PATH_IMAGES_VALID.format(id=img_id_c)
mask_path = DATA_PATH_MASKS_VALID.format(id=img_id_c)

img_valid_c  = np.load(img_path)
mask_valid_c = np.load(mask_path)

monitor = Monitoring(
    model = model,
    layerTracking = None,
    validImage = img_valid_c,
    validMask  = mask_valid_c)

Since the task of this exercise is more complex and training is more time consuming. It can be good to use a __Checkpoint callback__, that can store checkpoints of the network during training in memory, and that can be re-loaded later on if necessary. For more details on the checkpoint callback of __tf.keras__, [see the documentation](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint).

In [ ]:
# prepare callback to store checkpoints of the model
checkpoint_path = os.getcwd() + os.sep + "checkpoint_baseline.hdf5"
checkpointer = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    monitor  = 'val_loss',
    verbose  = 0,
    
    save_best_only = True,
    mode = "min")

We can finally run the training procedure (including all the generators and callbacks we defined before.)

In [ ]:
nbEpochs  = 500
model.fit_generator(
    generator       = dataGenerator_train,
    steps_per_epoch = 10,
    epochs          = nbEpochs,

    validation_data  = dataGenerator_valid,
    validation_freq  = 5,

    verbose   = 1,
    callbacks = [monitor, checkpointer])

## Qualitatitve evaluation: visualizing some predictions
Before evaluating the model, we can check qualitatively that the trained model is doing a good job on hold-out images. Let's select a sample from the validation set and compare the predictions with the ground-truth contour masks.

In [ ]:
# first we use the test generator to create batches of test images
arbitrary_iteration_idx = 42
batch_valid_images, batch_valid_masks = dataGenerator_valid[arbitrary_iteration_idx]
batch_valid_images = batch_valid_images[:8] # Selection of a small sample
batch_valid_masks  = batch_valid_masks[:8]

# then we get the prediction of the mode
tensor_predictions = model.predict(
    x = batch_valid_images)

# we format the results for visualization
batch_valid_images = batch_valid_images[:,:,:,0]
tensor_predictions = tensor_predictions[:,:,:,0]

print("Validation Images and Predicted Masks")
plot_image_batch(batch_valid_images, tensor_predictions) #-- We finally visualize the results

#print("Validation Images and Ground-Truth Masks")
#plot_image_batch(batch_valid_images, batch_valid_masks) #-- We finally visualize the results

## Visualizing the intermediate feature maps
To get an intuition of the learned features, intermediate feature maps can be visualized by defining an auxiliary model.

In [ ]:
# we define an auxillary model

model_aux = tf.keras.Model(inputs, x1_skip) # first conv layer as a target
#model_aux = tf.keras.Model(inputs, x2_skip) # second conv layer as a target
#model_aux = tf.keras.Model(inputs, x3_skip) # third conv layer as a target

In [ ]:
# we first select a random training image
img_id_c  = train_imageIds_list[np.random.randint(len(train_imageIds_list))]
img_path  = DATA_PATH_IMAGES_TRAIN.format(id=img_id_c)

demo_image = np.load(img_path)
demo_image = demo_image.astype(np.float) / 255.0 # [0,1] rescaling

demo_image = np.expand_dims(demo_image, axis=0) # enforce the shape of a batch of 1 image
demo_image = np.expand_dims(demo_image, axis=3) # enforce the shape of a batch of 1 image

In [ ]:
# we feed the selected image to the auxillary network
featureMaps = model_aux.predict(demo_image) # returns a tensor with shape [1,Height,Width,Channels]

plot_featureMaps(demo_image, featureMaps, maxNbChannels = 6)

## Quantitative evaluation on the validation Set
Now that we have a trained model, we would like to evaluate its performance on the validation set that was not used to train the model.
A metric commonly used to measure the difference between two binary maps is the [DICE coefficient](https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient) (also called $F_1$-score). The metric we will use in this exercise is the mean DICE coefficient across all the instances of the validation set.


Note that the raw generated predicted mask are a pixel-wise class label likelihood and are continuous values in \[0,1\]. Therefore we need to binarise the prediction in order to be compared (and the DICE coefficient is defined for 2 binary maps). A straight-forward solution is to threshold the network output with a fixed threshold.

In [ ]:
prediction_threshold = 0.9 # manually defined binarization threshold

### Evaluation on a single image

In [ ]:
def DICE(maskA, maskB):
    # make sure masks are binary maps
    maskA = maskA.astype(np.bool)
    maskB = maskB.astype(np.bool)
    
    # DICE coefficient definition: A ^ B / (|A| + |B|)
    interAB = maskA * maskB
    score   = 2 * np.sum(interAB) / (np.sum(maskA) + np.sum(maskB))
    return score

def pad_mask(source, target):
    target_h, target_w   = target.shape
    source_h, source_w   = source.shape
    pad_y = (target_h - source_h) // 2
    pad_x = (target_w - source_w) // 2
    
    return np.pad(source, ([pad_y,pad_y], [pad_x,pad_x]))

def import_valid_data(valid_id):
    # Import the image
    img_path  = DATA_PATH_IMAGES_VALID.format(id=valid_id)
    img_c = np.load(img_path) / 255.0

    # Import the ground-truth mask
    mask_path  = DATA_PATH_MASKS_VALID.format(id=valid_id)
    mask_c = np.load(mask_path)  
    
    return img_c, mask_c

# select the id of a validation data point
random_index = np.random.randint(valid_imageIds_list.shape[0])
img_id = valid_imageIds_list[random_index] 

img_c, mask_true = import_valid_data(img_id)

# extend to fit the input placeholder of the network
img_c_ext = np.expand_dims(img_c, axis=0)
img_c_ext = np.expand_dims(img_c_ext, axis=3)

# compute the prediction
mask_pred = model.predict(
     x = img_c_ext)
mask_pred = mask_pred[0,:,:,0] # Remove undesired dimensions

# pad the predicted mask to match the input dimensions
mask_pred = pad_mask(mask_pred, mask_true)

# binarize the predicted mask
mask_pred_bin = mask_pred > prediction_threshold

plot_image_list(
    [mask_true, mask_pred_bin, mask_true * mask_pred_bin],
    ["True Contour", "Predicted Contour", "Intersection"])

# compute DICE SCORE
dice = DICE(mask_true, mask_pred_bin)
print("Dice score:", dice)

### Evaluation on the full validation set
Let's compute the mean DICE coefficient on the validation set by scanning all its data points *(this can take a few minutes)*.

In [ ]:
def DICE(maskA, maskB):
    # make sure masks are binary maps
    maskA = maskA.astype(np.bool)
    maskB = maskB.astype(np.bool)
    
    # DICE coefficient definition: A ^ B / (|A| + |B|)
    interAB = maskA * maskB
    score   = 2 * np.sum(interAB) / (np.sum(maskA) + np.sum(maskB))
    return score

def pad_mask(source, target):
    target_h, target_w   = target.shape
    source_h, source_w   = source.shape
    pad_y = (target_h - source_h) // 2
    pad_x = (target_w - source_w) // 2
    
    return np.pad(source, ([pad_y,pad_y], [pad_x,pad_x]))

def import_valid_data(valid_id):
    # import the image
    img_path  = DATA_PATH_IMAGES_VALID.format(id=valid_id)
    img_c = np.load(img_path) / 255.0

    # import the ground-truth mask
    mask_path  = DATA_PATH_MASKS_VALID.format(id=valid_id)
    mask_c = np.load(mask_path)  
    
    return img_c, mask_c

score_list = []
for idx_scan, img_id in enumerate(valid_imageIds_list):
    img_c, mask_true = import_valid_data(img_id)

    # extend to fit the input placeholder of the network
    img_c_ext = np.expand_dims(img_c, axis=0)
    img_c_ext = np.expand_dims(img_c_ext, axis=3)


    # compute the prediction
    mask_pred = model.predict(
         x = img_c_ext)
    mask_pred = mask_pred[0,:,:,0] # Remove undesired dimensions

    # pad the predicted mask to match the input dimensions
    mask_pred = pad_mask(mask_pred, mask_true)

    # binarize the predicted mask
    mask_pred_bin = mask_pred > prediction_threshold

    # compute DICE SCORE
    dice = DICE(mask_true, mask_pred_bin)
    
    score_list.append(dice)
    
    print("[Scan validation set {}/{}] Running average DICE {}".format(idx_scan+1, valid_imageIds_list.shape[0], np.mean(score_list)), end="\r")

print("\nAverage DICE:", np.mean(score_list))

# PART 2: IMPROVING THE MODEL AND MINI-CHALLENGE SUBMISSION
Now that you have completed PART 1 of the exercise, you should have a baseline that produces reasonable performance, but by no means perfect.

In this PART 2, your goal is to expend and improve this baseline model. Below is a list of suggestions for improvements.

## SUGGESTIONS FOR IMPROVEMENT
- Investigating other pre-processing methods (by changing the generator so as to apply a normalization/whitening transformation on the network inputs).
- Investigating other data-sampling approaches (by changing the generator so that inputs follow another sampling-distribution. For instance a patient-based distribution might be more realistic and more likely to generalize).
- Investigating different model architecture (by changing the network hyperparameters, adding layers 
etc. See (batchNorm layers)[https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization].)
- Investigating other optimizers and parametrizations (see (learning rate schedulling)[https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LearningRateScheduler])
- Investigating other regularizers (see (tf.keras.regularizers)[https://www.tensorflow.org/api_docs/python/tf/keras/regularizers])
- Investigating post-processing approaches (ideally, the model output should be optimal but sometimes transforms can be found to refine the predictions).
- Searching optimal hyperparamters (for instance, you can write a script to find the optimum binarization threshold on the test set and use it on the test set).
- ... 

## MINI-CHALLENGE DESCRIPTION
Towards the end of the session you can run your best model on the __test set__ and make a submission for the mini-challenge via the __submit_results()__ function.
At the end of the session, the organizers will evaluate the participants submissions on the hold-out ground-truth contours mask and will release the results.


---

## Qualitative evaluation on the test set

To get a qualitative idea of the predictions made on the test set, you can visualize the predictions made for a whole CMR dynamic sequence using the function __plot_image_sequence()__.

In [ ]:
# Let choose the id of a specific patient
print("List of test patient ids: ", np.unique(test_metadata["patient_id"]))
selected_patient_id = 42
patient_indices = np.where(test_metadata["patient_id"] == selected_patient_id)[0]

# Select the 30 first images for this patient (they are already ordered)
image_indices   = test_imageIds_list[patient_indices][:30] 

# Import the images and create a tensor to be fed to the network
batch_images_test = []
for idx_c in image_indices:
    img_path  = DATA_PATH_IMAGES_TEST.format(id=idx_c)
    batch_images_test.append(np.load(img_path))

# Format the tensor
batch_images_test = np.stack(batch_images_test)
batch_images_test = batch_images_test / 255.0 # [0,1] rescaling
batch_images_test = np.expand_dims(batch_images_test, axis=3)

# Feed the tensor to the model
tensor_predictions = model.predict(
    x = batch_images_test)

# Finally we generate an animation of the image sequence, next to the predicted segmentation mask
stopAnimationFunction = plot_image_sequence(batch_images_test[:,:,:,0], tensor_predictions[:,:,:,0])


<span style="font-size:15pt;color:red"> *To prevent the notebook from crashing due to the animation, you can stop the animation by executing the next cell.* </span>

In [ ]:
stopAnimationFunction() # Stop the animation!

## Code to evaluate the full test set and submit the results
Let's scan the whole test set and compute predictions (this can take a few minutes), and then submit the compiled results (and wait for a confirmation message).

In [ ]:
def pad_mask(source, target):
    target_h, target_w   = target.shape[:2]
    source_h, source_w   = source.shape[:2]
    pad_y = (target_h - source_h) // 2
    pad_x = (target_w - source_w) // 2
    
    return np.pad(source, ([pad_y,pad_y], [pad_x,pad_x]))

def import_test_data(test_id):
    # Import the image
    img_path  = DATA_PATH_IMAGES_TEST.format(id=test_id)
    img_c = np.load(img_path) / 255.0
    
    return img_c

test_result_dict = {}
for idx_scan, img_id in enumerate(test_imageIds_list[:]):
    img_c = import_test_data(img_id)

    # Extend to fit the input placeholder of the network
    img_c_ext = np.expand_dims(img_c, axis=0)
    img_c_ext = np.expand_dims(img_c_ext, axis=3)

    # Compute the prediction
    mask_pred = model.predict(
         x = img_c_ext)
    mask_pred = mask_pred[0,:,:,0] # Remove undesired dimensions

    # Pad the predicted mask to match the input dimensions
    mask_pred = pad_mask(mask_pred, img_c)
    
    # Binarize the predicted mask
    prediction_threshold = 0.8
    mask_pred_bin = mask_pred > prediction_threshold
    
    test_result_dict[img_id] = mask_pred_bin
    
    print("[Scanning test set {}/{}]".format(idx_scan+1, test_imageIds_list.shape[0]), end="\r")

print("\n Test results, ready are to submit:", len(test_result_dict))

In [ ]:
# Finally the dictionary of results can be submitted (wait for confirmation)
received_results = submit_results(test_result_dict)

In [ ]:
# Quick test to check that binary mask were correctly submitted.
ex_mask = received_results["4220"]
plot_image_list(
    [ex_mask],
    ["Checking Test Prediction"])